- 로컬 환경에서 PDF 검색하기 1단계 step1
- PDF 문서를 로드하고 한국어 임베딩 모델을 적용하여 벡터 DB 생성
- 쿼리를 날려 검색하기까지의 과정

### 사용한 임베딩 모델 jhgan/ko-sroberta-multitask

In [6]:
# 라이브러리 설치 (필요한 경우 실행)
%pip install -U langchain langchain_core sentence-transformers faiss-cpu pymupdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  # 올바른 모듈에서 Embeddings 임포트
from sentence_transformers import SentenceTransformer

# Step 1: 문서 로드
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [8]:
# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
faiss_index = FAISS.from_documents(split_documents, embedding_model)

In [9]:
# Step 5: 유사 문서 검색 테스트
query = "AI 기술의 최신 동향"
query_embedding = embedding_model.embed_query(query)
results = faiss_index.similarity_search_by_vector(query_embedding, k=3)


In [10]:
# Step 6: 검색 결과 출력
for result in results:
    print(result.page_content)

Ⅰ. 인공지능 산업 동향 브리프
보장하고 책임성을 강화
∙산업계, 정부, 시민사회, 학계를 포함해 첨단 AI 시스템을 개발하는 조직 간 정보공유와 사고 발생 시 
신고를 위해 협력하고, 위험 기반 접근방식을 토대로 개인정보보호 정책과 위험 완화 조치를 포함하는 
AI 거버넌스와 위험 관리 정책을 마련
∙AI 수명주기 전반에 걸쳐 물리보안, 사이버보안, 내부자 위협 보안을 포함한 강력한 보안 통제 구현
∙사용자가 AI 생성 콘텐츠를 식별할 수 있도록 워터마크를 비롯하여 기술적으로 가능한 기법으로 
신뢰할 수 있는 콘텐츠 인증과 출처 확인 메커니즘을 개발 및 구축 
∙사회적 위험과 안전·보안 문제를 완화하는 연구와 효과적인 완화 대책에 우선 투자하고, 기후 위기 
대응, 세계 보건과 교육 등 세계적 난제 해결을 위한 첨단 AI 시스템을 우선 개발
∙국제 기술 표준의 개발 및 채택을 가속화하고, 개인정보와 지식재산권 보호를 위해 데이터 입력과 수집 
시 적절한 보호 장치 구현
☞ 출처: G7, Hiroshima Process International Code of Conduct for Advanced AI Systems, 2023.10.30.
n 연구진은 상기 원칙에 따라 AI를 성능에 따라 0~5단계와 광범위한 목적에 활용될 수 있는 범용 AI 및 특정 
과업에 활용되는 특수 AI로 분류했으며, 특수 AI에서는 5단계까지 달성되었으나, 범용 AI는 현재 1단계 수준
성능
특수 AI 예시
범용 AI 예시
0단계: AI 아님
계산기 소프트웨어, 컴파일러
아마존 메커니컬 터크
1단계: 신진(숙련되지 않은 인간)
GOFAI(Good Old Fashioned Artificial Intelligence) 
챗GPT, 바드, 라마2
2단계: 유능(숙련된 인간의 50% 이상)
스마트 스피커(애플 시리, 아마존 알렉사, 구글 
어시스턴트), IBM 왓슨 
미달성
3단계: 전문가(숙련된 인간의 90% 이상)
문법 교정기(그래머리), 생성 이미지 모델(달리2)
미달성
4단계: 거장(